<a href="https://colab.research.google.com/github/jaekim3220/jupyter/blob/main/15_CNN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>